In [ ]:
import pandas as pd
import sqlite3

In [ ]:
im_conn = sqlite3.connect("../data/im.db")

In [ ]:
bom_df = pd.read_csv("../zippedData/bom.movie_gross.csv.gz")
bom_df.head()

In [ ]:
movieinfo_df = pd.read_csv("../zippedData/rt.movie_info.tsv.gz", sep='\t', compression='gzip')
movieinfo_df.head()

In [ ]:
reviews_df = pd.read_csv(
    "../zippedData/rt.reviews.tsv.gz",
    sep='\t',
    compression='gzip',
    encoding='latin1'   # or encoding='ISO-8859-1'
)
reviews_df.head()


In [ ]:
tmdb_df = pd.read_csv("../zippedData/tmdb.movies.csv.gz")
tmdb_df.head()

In [ ]:
budgets_df = pd.read_csv("../zippedData/tn.movie_budgets.csv.gz")
budgets_df.head()

Basic Cleaning

In [ ]:
# check shape and missing values for all datasets

datasets = {
    "BOM": bom_df,
    "Movie Info": movieinfo_df,
    "Reviews": reviews_df,
    "TMDB": tmdb_df,
    "Budgets": budgets_df
}

for name, df in datasets.items():
    print(f"\n{name} dataset: {df.shape[0]} rows, {df.shape[1]} columns")
    print("Missing values summary:")
    display(df.isnull().sum().sort_values(ascending=True).head(10))


In [ ]:
#drop duplicates

for name, df in datasets.items():
    df.drop_duplicates(inplace=True)


In [ ]:
# this helps you identify if numeric columns like budgets or grosses are stored as text

for name, df in datasets.items():
    display(df.dtypes)



In [ ]:
# Remove $ and commas, convert to integer

for col in ["production_budget", "domestic_gross", "worldwide_gross"]:
    budgets_df[col] = (budgets_df[col].replace('[\$,]', '', regex=True).astype(float))

budgets_df[["production_budget", "domestic_gross", "worldwide_gross"]].head()


In [ ]:
# Make column names consistent across datasets; all lowercase, no spaces.

bom_df.columns = bom_df.columns.str.lower().str.replace(" ", "_")

movieinfo_df.columns = movieinfo_df.columns.str.lower().str.replace(" ", "_")

reviews_df.columns = reviews_df.columns.str.lower().str.replace(" ", "_")

tmdb_df.columns = tmdb_df.columns.str.lower().str.replace(" ", "_")

budgets_df.columns = budgets_df.columns.str.lower().str.replace(" ", "_")



Loading Cleaned Data into SQLite

In [ ]:
# Create a new database (or connect if exists)
conn = sqlite3.connect("../data/movies_cleaned.db")

# Save each dataframe as a SQL table

bom_df.to_sql("bom_gross", conn, if_exists="replace", index=False)

movieinfo_df.to_sql("rt_movie_info", conn, if_exists="replace", index=False)

reviews_df.to_sql("rt_reviews", conn, if_exists="replace", index=False)

tmdb_df.to_sql("tmdb_movies", conn, if_exists="replace", index=False)

budgets_df.to_sql("movie_budgets", conn, if_exists="replace", index=False)

print("Cleaned datasets loaded into SQLite database successfully!")


In [ ]:
# verification of tables in SQLite

pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)


In [ ]:
pd.read_sql("PRAGMA table_info(rt_movie_info);", conn)


In [ ]:
for table in ["bom_gross", "rt_reviews", "tmdb_movies", "movie_budgets"]:
    print(f"\n{table} columns:")
    display(pd.read_sql(f"PRAGMA table_info({table});", conn))


OBJECTIVE 1: Identify High-Performing Studios & Genres

We start by finding which studios consistently produce the highest grossing movies. 
 
The `bom_gross` and `movie_budgets` tables are used here.

In [ ]:
query_studio_performance = """
SELECT
    bg.studio,
    ROUND(AVG(mb.worldwide_gross), 2) AS avg_worldwide_gross,
    COUNT(mb.movie) AS num_movies
FROM movie_budgets mb
JOIN bom_gross bg
    ON mb.movie = bg.title
GROUP BY bg.studio
HAVING num_movies > 3
ORDER BY avg_worldwide_gross DESC
LIMIT 10;
"""

studio_performance_df = pd.read_sql(query_studio_performance, conn)
studio_performance_df


OBJECTIVE 2: Analyzing ROI (Profitability)

This calculate each movie’s ROI to identify which films and studios achieve the best returns.


In [ ]:
query_roi = """
SELECT
    mb.movie,
    bg.studio,
    ROUND((mb.worldwide_gross - mb.production_budget) / mb.production_budget, 2) AS ROI,
    mb.worldwide_gross,
    mb.production_budget
FROM movie_budgets mb
JOIN bom_gross bg
    ON mb.movie = bg.title
WHERE mb.production_budget > 0
ORDER BY ROI DESC
LIMIT 10;
"""

roi_df = pd.read_sql(query_roi, conn)
roi_df


OBJECTIVE 3: Audience Ratings and Popularity

This shows which genres and types of films receive high audience ratings and votes using TMDb data.


In [ ]:
query_genre_ratings = """
SELECT
    rmi.genre AS genre,
    ROUND(AVG(tm.vote_average), 2) AS avg_rating,
    COUNT(*) AS num_movies
FROM tmdb_movies tm
JOIN movie_budgets mb
    ON tm.title = mb.movie
JOIN rt_movie_info rmi
    ON rmi.studio = mb.movie OR rmi.genre IS NOT NULL
GROUP BY rmi.genre
HAVING num_movies > 5
ORDER BY avg_rating DESC
LIMIT 10;
"""

genre_ratings_df = pd.read_sql(query_genre_ratings, conn)
genre_ratings_df



OBJECTIVE 4: Movie Performance Trends Over Time

We’ll explore whether movie performance has improved or declined over time, based on box office and budgets.


In [ ]:
query_trends = """
SELECT
    bg.year,
    ROUND(AVG(mb.worldwide_gross), 2) AS avg_gross,
    ROUND(AVG(mb.production_budget), 2) AS avg_budget,
    COUNT(*) AS num_movies
FROM movie_budgets mb
JOIN bom_gross bg
    ON mb.movie = bg.title
GROUP BY bg.year
HAVING num_movies > 5
ORDER BY bg.year ASC;
"""

trends_df = pd.read_sql(query_trends, conn)
trends_df.head()


OBJECTIVE 5: Identify Top Performing Movies

This involves finding the most profitable and highest rated movies across all sources.


In [ ]:
query_best_movies = """
SELECT
    mb.movie,
    bg.studio,
    tm.vote_average AS rating,
    ROUND((mb.worldwide_gross - mb.production_budget) / mb.production_budget, 2) AS ROI,
    mb.worldwide_gross
FROM movie_budgets mb
JOIN bom_gross bg
    ON mb.movie = bg.title
JOIN tmdb_movies tm
    ON mb.movie = tm.title
WHERE mb.production_budget > 0
ORDER BY ROI DESC, rating DESC
LIMIT 10;
"""

best_movies_df = pd.read_sql(query_best_movies, conn)
best_movies_df


**SQL INSIGHTS**


**Top Studios:** The most successful studios generate high worldwide grosses consistently.  

**ROI Leaders:** Low-budget, high-grossing films show strong profit potential.  

**Ratings:** Some genres (from TMDb `genre_ids`) correlate with higher average audience ratings. 

**Trends:** Movie budgets and grosses have shifted over years, showing changing audience interests.  

**Top Titles:** Combining ROI and ratings highlights films that are both profitable and popular.

## Data Cleaning<hr>
In this section, we do the final cleaning the data from the data sources

#### 1 BOM DF

In [ ]:
# Load bom_gross
bbom_df = pd.read_sql('''
 SELECT * FROM bom_gross;
''', conn)

In [ ]:
bbom_df.head()

In [ ]:
bbom_df.describe()

In [ ]:
bbom_df.info()

In [ ]:
(2037/3387)*100

In [ ]:
bbom_df["studio"].value_counts()

In [ ]:
bbom_df[bom_df["studio"].isna()]

In [ ]:
bbom_df[bom_df["studio"]=="NotSpecified"]

In [ ]:
bbom_df["studio"].fillna("NotSpecified", inplace=True)

In [ ]:
bbom_df[bom_df["domestic_gross"].isna() & bom_df["foreign_gross"].isna()]

In [ ]:
bbom_df[bom_df["domestic_gross"].isna()]

In [ ]:
bbom_df.dropna(subset=["domestic_gross", "foreign_gross"], inplace=True)

We've decided to drop the columns that have missing revenue numbers since we want accurate numbers and filling in with mean or median may inflate or deflate some films hence giving us wrong insights

In [ ]:
bbom_df.info()

In [ ]:
#Connecting to cleaned movies
cleaned_conn = sqlite3.connect("../cleaned_data/movies.db")

In [ ]:
# Saving to cleaned database
bbom_df.to_sql("bom_gross", cleaned_conn, if_exists="replace", index=False)

#### 2. RT Movie Info

In [ ]:
# Load RT Movie info
bmovieinfo_df = pd.read_sql('''
 SELECT * FROM rt_movie_info
''', conn)

In [ ]:
bmovieinfo_df.head()

In [ ]:
#Describe
bmovieinfo_df.info()

In [ ]:
# Checking for all the currencies in the table
bmovieinfo_df["currency"].value_counts()

We are dropping currency columns since they are all in dollars hence it is a redudant column

In [ ]:
bmovieinfo_df.drop(columns="currency", inplace=True)

In [ ]:
bmovieinfo_df

In [ ]:
bmovieinfo_df[bmovieinfo_df["box_office"].isna() == False]["box_office"]

In [ ]:
# Marking missing box_office_missing since it's an important column even though most values are missing
# This makes it easier to filter out filled in values
bmovieinfo_df['box_office_missing'] = bmovieinfo_df['box_office'].isna()

In [ ]:
#bmovieinfo_df.drop(columns="box_office_missing", inplace=True)

In [ ]:
bmovieinfo_df.head(10)

In [ ]:
# Remove commas
bmovieinfo_df['box_office'] = (
    bmovieinfo_df['box_office']
    .replace('None', pd.NA)        
    .str.replace(',', '', regex=True)  # remove commas
)


In [ ]:
# Convert box_office to numeric data type
bmovieinfo_df["box_office"] = pd.to_numeric(bmovieinfo_df["box_office"], errors="coerce")

In [ ]:

bmovieinfo_df.head(10)

In [ ]:
bmovieinfo_df.describe()

In [ ]:
rt_info_bo_median = bmovieinfo_df["box_office"].median()
rt_info_bo_median

In [ ]:
# Filling missing box office values with the box_office column median
bmovieinfo_df["box_office"].fillna(rt_info_bo_median,inplace=True)

In [ ]:
bmovieinfo_df.head(10)

We have handled the box office column by marking missing values as missing and then filling them with median

In [ ]:
bmovieinfo_df[bmovieinfo_df["director"].isna()]

In [ ]:
bmovieinfo_df["runtime"].head()

In [ ]:
bmovieinfo_df['runtime'].str.replace('minutes', '', regex=True)  # remove commas

In [ ]:
bmovieinfo_df['runtime'] = (
    bmovieinfo_df['runtime']
    .str.replace('minutes', '', regex=True)  # remove commas
)


bmovieinfo_df["box_office"] = pd.to_numeric(bmovieinfo_df["box_office"], errors="coerce")

In [ ]:
bmovieinfo_df["runtime"] = pd.to_numeric(bmovieinfo_df["runtime"], errors="coerce")

In [ ]:
bmovieinfo_df[bmovieinfo_df['runtime'].isna()]

In [ ]:
bmovieinfo_df["runtime"].describe()

In [ ]:
bmvinf_run_mean = bmovieinfo_df['runtime'].mean()
bmvinf_run_mean

In [ ]:
bmovieinfo_df['runtime'].fillna(bmvinf_run_mean, inplace = True)

In [ ]:
bmovieinfo_df.info()

In [ ]:

# Saving to cleaned database
bmovieinfo_df.to_sql("rt_movie_info", cleaned_conn, if_exists="replace", index=False)

#### 3. RT Reviews

In [ ]:
# Loading RT Reviews
breview_df = pd.read_sql('''
    SELECT * FROM rt_reviews;
''', conn) 

breview_df.head(10)

In [ ]:
breview_df.info()

I'm dropping rows with missing ratings since they are of no use to as

In [ ]:
breview_df.dropna(subset=["rating"], inplace=True)

In [ ]:
breview_df

In [ ]:
breview_df['rating'].unique()

In [ ]:
# Saving to cleaned database
breview_df.to_sql("rt_reviews", cleaned_conn, if_exists="replace", index=False)

#### 5. TMDB 

In [ ]:
# Loading the tmdb database
btmdb_df = pd.read_sql('''
    SELECT * FROM tmdb_movies;
''',conn)

In [ ]:
btmdb_df

In [ ]:
#Pandas is showing duplicate index columns so we drop one in the following 2 columns
btmdb_df = btmdb_df.rename(columns={"unnamed:_0": "index"})

In [ ]:
btmdb_df.iloc

In [ ]:
btmdb_df = btmdb_df.set_index("index")

In [ ]:
btmdb_df

In [ ]:
#looking at tmdb metadata
btmdb_df.info()

No need for cleaning

In [ ]:
btmdb_df.describe()

In [ ]:
# Saving to cleaned database
btmdb_df.to_sql("tmdb_movies", cleaned_conn, if_exists="replace", index=False)

#### 5. Budgets

In [ ]:
# Loading budget
bbudgets_df = pd.read_sql('''
    SELECT * FROM movie_budgets
''',conn)

In [ ]:
bbudgets_df

In [ ]:
#Checking for descriptive stats
bbudgets_df.describe()

In [ ]:
#Checking for null values
bbudgets_df.info()

No need for cleaning

In [ ]:
# Saving to cleaned database
bbudgets_df.to_sql("movie_budgets", cleaned_conn, if_exists="replace", index=False)

# Hypothesis

In [ ]:
query_ratings = """
SELECT
    rmi.genre AS genre,
    ROUND(AVG(tm.vote_average), 2) AS avg_rating,
    COUNT(*) AS num_movies
FROM tmdb_movies tm
JOIN movie_budgets mb
    ON tm.title = mb.movie
JOIN rt_movie_info rmi
    ON rmi.studio = mb.movie OR rmi.genre IS NOT NULL
GROUP BY rmi.genre
HAVING num_movies > 5
ORDER BY avg_rating DESC
LIMIT 10;
"""

bratings_df = pd.read_sql(query_ratings, cleaned_conn)
bratings_df

In [ ]:
bratings_df.info()

In [ ]:
query_roi = """
SELECT
    mb.movie,
    bg.studio,
    ROUND((mb.worldwide_gross - mb.production_budget) / mb.production_budget, 2) AS ROI,
    mb.worldwide_gross,
    mb.production_budget
FROM movie_budgets mb
JOIN bom_gross bg
    ON mb.movie = bg.title
WHERE mb.production_budget > 0
ORDER BY ROI DESC
LIMIT 10;
"""

broi_df = pd.read_sql(query_roi, cleaned_conn)
broi_df


In [ ]:
def hypothesis_test(ho,h1,p_value,alpha=0.05):
    if p_value < alpha:
        print(f"Reject the null hypothesis: {h0}")
        print(f"Accept the alternative hypothesis: {h1}")
    else:
        print(f"Fail to reject the null hypothesis: {h0}")
        print(f"Fail to accept the alternative hypothesis: {h1}")

In [ ]:
from scipy.stats import f_oneway
# grouping movie studios by ROI
groups=broi_df.groupby('studio')['ROI'].apply(list)  
    #run ANOVA
f_stat,p_value=f_oneway(*groups)
print(f"f_statistic:",f_stat)
print(f"p_value:",p_value)

In [ ]:
h0="there is no difference in ROI between studios."
h1="there is a difference in ROI between studiods."
print(f"F-statistic={f_stat}, p-value={p_value}")
hypothesis_test(h0,h1,p_value)

In [ ]:
#Closing database connections
""" 
WARNING! THIS SHOULD BE THE LAST CELL TO BE RAN SO AS TO AVOID ERRORS
"""
conn.close()
cleaned_conn.close()
im_conn.close()